## 7 制約充足問題の例



Google Colabの場合は最初に以下を実行してください．



In [1]:
! apt update && apt install openjdk-8-jre-headless
! git clone https://github.com/tamura70/cspsat-jupyter.git
%cd cspsat-jupyter

### 7.1 魔方陣



例題「3x3の魔方陣」のCNF式



In [1]:
from cspsat import *
def magicSquare3x3(x=Var("x")):
    for i in range(3):
        for j in range(3):
            yield ["int", x(i,j), 1, 9]
    xx = [ x(i,j) for i in range(3) for j in range(3) ]
    yield ["alldifferent", *xx]
    for i in range(3):
        yield ["==", ["+", x(i,0), x(i,1), x(i,2)], 15]
    for j in range(3):
        yield ["==", ["+", x(0,j), x(1,j), x(2,j)], 15]
    yield ["==", ["+", x(0,0), x(1,1), x(2,2)], 15]
    yield ["==", ["+", x(0,2), x(1,1), x(2,0)], 15]

In [1]:
for c in magicSquare3x3():
    print(c)

In [1]:
solveCSP(magicSquare3x3())

In [1]:
solveCSP(magicSquare3x3(), num=0)

#### 練習問題 7.1.2: $n$×$n$の魔方陣



魔方陣のCSP



In [1]:
from cspsat import *
def magicSquare(n, x=Var("x")):
    s = n*(n**2+1)//2
    xx = [ x(i,j) for i in range(n) for j in range(n) ]
    for v in xx:
        yield ["int", v, 1, n*n]
    yield ["alldifferent", *xx]
    for i in range(n):
        yield ["==", ["+", *[ x(i,j) for j in range(n) ]], s]
    for j in range(n):
        yield ["==", ["+", *[ x(i,j) for i in range(n) ]], s]
    yield ["==", ["+", *[ x(i,i) for i in range(n) ]], s]
    yield ["==", ["+", *[ x(i,n-i-1) for i in range(n) ]], s]
    yield ["<", x(0,0), x(0,n-1)]
    yield ["<", x(0,0), x(n-1,0)]
    yield ["<", x(0,0), x(n-1,n-1)]
    yield ["<", x(0,n-1), x(n-1,0)]

In [1]:
n = 5
x = Var("x")
for sol in solutionsCSP(magicSquare(n)):
    for i in range(n):
        print([ sol[x(i,j)] for j in range(n) ])

#### 練習問題 7.1.3: $n$×$n$の汎対角魔方陣



汎対角魔方陣



In [1]:
from cspsat import *
def pandiagonalMagicSquare(n, x=Var("x")):
    s = n*(n**2+1)//2
    xx = [ x(i,j) for i in range(n) for j in range(n) ]
    for v in xx:
        yield ["int", v, 1, n*n]
    yield ["alldifferent", *xx]
    for i in range(n):
        yield ["==", ["+", *[ x(i,j) for j in range(n) ]], s]
    for j in range(n):
        yield ["==", ["+", *[ x(i,j) for i in range(n) ]], s]
    for a in range(n):
        yield ["==", ["+", *[ x(i,(a-i)%n) for i in range(n) ]], s]
    for b in range(n):
        yield ["==", ["+", *[ x(i,(i-b)%n) for i in range(n) ]], s]
    yield ["<", x(0,0), x(0,n-1)]
    yield ["<", x(0,0), x(n-1,0)]
    yield ["<", x(0,0), x(n-1,n-1)]
    yield ["<", x(0,n-1), x(n-1,0)]

In [1]:
n = 4
x = Var("x")
for sol in solutionsCSP(pandiagonalMagicSquare(n), num=0):
    for i in range(n):
        print([ sol[x(i,j)] for j in range(n) ])
    print()

### 7.2 数独



数独のCSP (sudoku1)



In [1]:
from cspsat import *
from itertools import combinations, product
def alldiff(xx):
    for (xi,xj) in combinations(xx, 2):
        yield ["!=", xi, xj]
def sudoku1(clues, x=Var("x")):
    for (i,j) in product(range(9), range(9)):
        yield ["int", x(i,j), 1, 9]
    for i in range(9):
        yield from alldiff([ x(i,j) for j in range(9) ])
    for j in range(9):
        yield from alldiff([ x(i,j) for i in range(9) ])
    for (i,j) in product(range(0,9,3), range(0,9,3)):
        yield from alldiff([ x(i+a,j+b) for a in range(3) for b in range(3) ])
    for (i,j) in product(range(9), range(9)):
        if clues[i][j] > 0:
            yield ["==", x(i,j), clues[i][j]]

例題「数独1」の解 (sudoku1)



In [1]:
s1 = [
    [0,0,7,1,8,9,2,0,0], [0,9,0,0,0,0,0,6,0], [8,0,0,0,0,0,0,0,9],
    [0,1,0,0,0,0,0,0,0], [0,0,6,7,4,1,8,0,0], [0,0,0,0,0,0,0,2,0],
    [9,0,0,0,0,0,0,0,5], [0,2,0,0,0,0,0,3,0], [0,0,4,8,7,6,9,0,0]
]
x = Var("x")
for sol in solutionsCSP(sudoku1(s1), encoder="direct"):
    for i in range(9):
        print([ sol[x(i,j)] for j in range(9) ])

In [1]:
print(status())

例題「数独2」の解 (sudoku1)



In [1]:
s2 = [
    [0,0,0,0,0,0,0,0,0], [0,4,3,0,0,0,6,7,0], [5,0,0,4,0,2,0,0,8],
    [8,0,0,0,6,0,0,0,1], [2,0,0,0,0,0,0,0,5], [0,5,0,0,0,0,0,4,0],
    [0,0,6,0,0,0,7,0,0], [0,0,0,5,0,1,0,0,0], [0,0,0,0,8,0,0,0,0]
]
x = Var("x")
for sol in solutionsCSP(sudoku1(s2), encoder="direct"):
    for i in range(9):
        print([ sol[x(i,j)] for j in range(9) ])
    print(status())

数独のCSP (sudoku2)



In [1]:
def alldiff2(xx):
    yield from alldiff(xx)
    for k in range(1,10):
        yield ["or", *[ ["==", x, k] for x in xx ]]
def sudoku2(clues):
    x = Var("x")
    for (i,j) in product(range(9), range(9)):
        yield ["int", x(i,j), 1, 9]
    for i in range(9):
        yield from alldiff2([ x(i,j) for j in range(9) ])
    for j in range(9):
        yield from alldiff2([ x(i,j) for i in range(9) ])
    for (i,j) in product(range(0,9,3), range(0,9,3)):
        yield from alldiff2([ x(i+a,j+b) for a in range(3) for b in range(3) ])
    for (i,j) in product(range(9), range(9)):
        if clues[i][j] > 0:
            yield ["==", x(i,j), clues[i][j]]

In [1]:
solveCSP(sudoku2(s2), encoder="direct", stat=True)

数独のCSP (9×9)



In [1]:
from cspsat import *
from itertools import product
def sudoku(clues):
    x = Var("x")
    for (i,j) in product(range(9), range(9)):
        yield ["int", x(i,j), 1, 9]
    for i in range(9):
        yield ["alldifferent", *[ x(i,j) for j in range(9) ]]
    for j in range(9):
        yield ["alldifferent", *[ x(i,j) for i in range(9) ]]
    for (i,j) in product(range(0,9,3), range(0,9,3)):
        yield ["alldifferent", *[ x(i+a,j+b) for a in range(3) for b in range(3) ]]
    for (i,j) in product(range(9), range(9)):
        if clues[i][j] > 0:
            yield ["==", x(i,j), clues[i][j]]

In [1]:
solveCSP(sudoku(s2), encoder="direct", stat=True)

#### 練習問題 7.2.1: 世界一難しい数独



In [1]:
s3 = [
    [8,0,0,0,0,0,0,0,0], [0,0,3,6,0,0,0,0,0], [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0], [0,0,0,0,4,5,7,0,0], [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8], [0,0,8,5,0,0,0,1,0], [0,9,0,0,0,0,4,0,0]
]

In [1]:
x = Var("x")
for sol in solutionsCSP(sudoku(s3), encoder="direct", num=0):
    for i in range(9):
        print([ sol[x(i,j)] for j in range(9)])
    print(status())

#### 練習問題 7.2.2: ジャイアント数独



数独のCSP (n×n)



In [1]:
from cspsat import *
from itertools import product
def giantSudoku(clues, n=9, m=3, x=Var("x")):
    for (i,j) in product(range(n), range(n)):
        yield ["int", x(i,j), 1, n]
    for i in range(n):
        yield ["alldifferent", *[ x(i,j) for j in range(n) ]]
    for j in range(n):
        yield ["alldifferent", *[ x(i,j) for i in range(n) ]]
    for (i,j) in product(range(0,n,m), range(0,n,m)):
        yield ["alldifferent", *[ x(i+a,j+b) for a in range(m) for b in range(m) ]]
    for (i,j) in product(range(n), range(n)):
        if clues[i][j] > 0:
            yield ["==", x(i,j), clues[i][j]]

Clarity Media Ltd.の36×36ジャイアント数独



In [1]:
s4 = [
[20, 13,  0, 34,  7, 15, 16,  1,  0, 19,  0, 24,  4, 12,  0,  5,  0,  0,  0, 28, 35,  0, 14, 30, 21, 10,  8, 22,  0,  0, 32, 29, 11,  9,  0, 26],
[ 0, 17,  9,  0, 11,  0, 27,  0,  4,  0,  0, 15, 33,  7,  0,  0, 21,  0, 19,  0, 12, 25,  0,  0, 28,  0, 14, 24,  0, 23,  6, 16,  0, 13, 35, 36],
[27, 18,  0,  0,  8, 26,  0, 20,  0, 28, 23,  0,  0, 30, 11, 14,  1, 16,  0,  0, 33,  0,  6,  0, 34,  0,  3, 13,  0,  0, 25, 19,  5,  0, 31,  0],
[19,  0, 22, 16,  0, 24,  0,  5, 18, 32,  0,  8,  0, 15,  0,  0, 17,  6, 11,  0,  2,  0, 23,  0, 35,  0,  9, 30,  0, 36,  1,  0, 14,  0,  0, 28],
[33,  1,  2,  0,  4,  0,  0,  0, 14,  0,  9,  0, 32,  0,  0,  0, 19, 34,  8,  0, 31, 36,  0,  0,  0, 16, 11, 18,  5,  0,  0, 21,  0, 17, 20, 30],
[ 5, 35, 28, 14,  0,  0, 33, 30,  0,  0, 12, 31,  9,  8,  3,  0, 29, 13,  0, 21,  4,  0, 26,  0, 32,  6,  2,  0, 20,  0,  0,  0, 15, 22,  0, 27],
[ 0, 20,  0, 22,  0,  0,  0, 32, 25,  2, 29, 14, 28,  0, 26, 36,  0,  0,  0,  5, 24,  0, 21,  4,  0,  3,  0, 31,  9,  0,  8,  0,  0,  0, 17, 23],
[ 0,  0,  0,  0, 17, 34,  4, 35, 10,  3,  0, 13,  0,  6, 24,  7, 32,  0,  0, 16,  8,  0, 12, 18,  0, 26, 25, 28, 36, 22,  5,  0, 27, 14,  0, 20],
[26,  9, 27, 25, 36,  0, 34, 28,  0,  0,  0,  0,  0,  0,  0, 18, 11,  0,  0,  0, 20, 32,  0,  0,  0,  7,  4,  0, 23, 17,  0, 24, 31,  0, 21,  0],
[11, 32, 12, 18, 10,  4,  0, 16, 31,  0,  0, 17,  3, 25,  2, 15,  0, 20, 36,  1,  0, 35,  0,  0, 19,  0,  0,  0, 30, 24,  0,  0, 28, 33,  0,  7],
[ 6,  0,  0,  0, 24,  5, 20,  8, 11,  0,  0, 18, 12,  0, 35,  0,  0,  0, 27, 34,  0,  0, 28, 23, 10,  0,  0,  0,  0, 13, 36, 22,  0,  4,  0,  0],
[13,  8, 23, 21,  0, 16,  0,  0,  0, 30, 24, 36,  0, 22, 10, 17,  4,  1,  0, 31,  0, 14, 25,  0, 27, 34, 12,  0, 35, 32,  9,  0,  2,  0, 29,  3],
[16,  0,  0,  0,  0,  0,  5,  4,  0,  0, 31,  0, 30, 10,  0,  0,  0, 32, 33,  0,  0,  0,  0, 15,  0, 24, 35,  0,  0, 26,  2, 27,  0,  0,  1, 19],
[ 2,  0, 18, 26,  0, 21, 36, 24,  0,  0, 19, 27,  0,  0,  5,  0, 15, 22, 13, 32,  0, 16,  0, 12, 33,  0, 29,  0, 28,  0, 31,  0,  9, 34,  3, 10],
[ 0, 22,  0,  0, 33,  0,  0,  0,  2, 21,  0, 30,  0, 23,  9, 13,  8,  7,  0, 27, 19, 24,  3,  0, 16, 25, 10,  0, 18,  5, 29,  0,  0, 35,  0,  0],
[10, 19,  5, 23,  1,  3, 17, 13, 20,  0,  0,  0,  0,  0, 25, 28,  0, 11, 31,  0, 14, 30,  0,  0, 15,  0, 32,  6, 34, 21,  0,  0,  0, 24,  0, 22],
[ 8,  0,  0,  0,  0, 11, 18, 15,  0, 10, 28, 25,  0, 33, 19,  0, 36, 17,  0, 23,  0,  7, 34,  0, 13,  0,  0, 12, 27,  0, 30, 20, 16, 21, 14,  0],
[ 0, 15,  0, 17, 29,  0,  0,  0,  0,  1, 33,  0, 18, 24,  0, 16,  0, 31,  5, 10, 36, 20,  2, 21,  3,  0, 30,  0,  0,  0, 11, 32, 13, 28,  0,  0],
[ 0,  0, 24, 10,  5,  2,  0,  0,  0, 22,  0, 20, 14, 26, 33, 35, 28, 27,  3,  0, 30,  0, 17, 11,  0, 19, 34,  0,  0,  0,  0, 23, 29,  0, 32,  0],
[ 0, 34, 15, 36, 32, 35,  0,  7,  9,  0,  0, 19,  0,  2,  1,  0, 18,  0, 14, 29,  0, 21, 27,  0, 12,  5, 24,  0, 10, 16, 33,  0,  0,  0,  0, 13],
[14,  0, 16,  0,  0,  0, 30, 33, 27, 25,  0, 10,  0,  0, 31, 34,  0, 29,  4,  0,  5, 15,  0,  0,  0,  0,  0, 21, 32,  1, 28, 18,  3, 19,  2,  9],
[ 0,  0,  3,  0,  0,  1,  2, 36,  0, 34, 32, 23,  0, 21,  7, 11, 20,  0, 16, 26, 10,  9, 22,  0, 30,  0, 28, 25,  0,  0,  0, 14,  0,  0,  5,  0],
[22, 21, 26, 29,  0, 28,  0, 14,  0,  5,  0, 12, 16,  0, 30,  0, 23,  4, 32,  6,  0, 31,  0,  0, 20,  2,  0,  0, 17, 33, 27,  0, 24, 25,  0, 11],
[17,  4,  0,  0, 12,  8,  1,  0,  0, 24, 26,  0, 15,  0,  0,  0,  0,  5, 34,  0,  0,  0, 18, 33,  0, 31,  0,  0,  6,  9,  0,  0,  0,  0,  0, 35],
[ 1, 36,  0, 19,  0, 12, 29, 18,  0, 26, 35, 33,  0, 32,  8,  0,  5,  0, 20, 15, 27, 23, 13,  0, 25,  4,  6,  0,  0,  0, 17,  0, 22,  7, 24, 31],
[ 0,  0, 33,  0, 13,  6, 10,  0,  0,  0,  0,  9, 35, 28,  0,  0, 24, 18,  0,  0,  0,  8,  0, 14,  1,  0,  0,  5, 22, 20, 15, 36,  0,  0,  0, 29],
[29,  0, 34,  8,  0,  0, 24, 21,  0,  0,  0, 32,  0,  0, 15,  0,  6, 10, 28,  0,  9,  5, 33, 31,  7,  0,  0, 26,  3,  0, 16, 35, 12, 11, 23,  2],
[ 0,  5,  0,  7, 18,  0,  3, 11,  0,  8, 14,  0,  0,  0, 29, 21,  0,  0,  0, 22, 34,  0,  0,  0,  0,  0,  0,  0, 24, 31,  0, 10,  4,  6, 25, 32],
[ 3,  0, 10, 20,  0, 22,  6,  2,  7, 12, 17,  0, 31, 13,  0, 23, 30,  0,  0,  4, 18, 26, 24,  0, 11,  0, 16, 15, 29, 35, 14,  9,  0,  0,  0,  0],
[24, 26,  0,  0,  0, 31,  0, 23, 34,  0, 25,  0, 22,  3,  0, 12,  7,  0,  0,  0, 29,  6,  0,  1,  8, 14, 13,  9, 21,  0,  0,  0, 20,  0, 27,  0],
[35,  0, 20,  9,  0,  0,  0, 19,  0,  7, 27, 21,  0, 18,  0,  6, 10,  0, 29, 33,  0, 11, 16, 34, 22, 30,  0,  0, 12, 28,  0,  0, 25, 31,  8, 14],
[23, 12,  7,  0, 15,  0,  0,  9, 16,  4,  1,  0,  0,  0, 32, 27,  0, 25,  6, 30,  0,  0,  0, 17,  0, 18,  0, 35,  0,  0,  0, 11,  0, 29, 10, 21],
[36,  0,  0,  5,  0, 19, 32,  0, 23, 18,  0, 34,  0,  1,  0, 30,  0, 24, 25, 35,  0,  0, 31,  0,  9,  0,  7, 10,  8,  0, 20,  0, 33, 27,  0, 16],
[ 0, 10,  0,  6, 34, 18,  0,  0, 30, 14,  0, 26,  0, 29,  0, 33,  0,  0, 23, 19, 15, 27,  8,  0,  0, 11,  5,  0,  1,  0, 24, 12,  0,  0,  9, 17],
[28, 16,  4,  0,  3, 13,  8,  0, 29, 33,  0, 11,  0,  0, 20, 31,  0,  9,  0, 14,  0,  0,  5, 10, 24,  0,  0, 27,  0, 25,  0,  6,  0, 30,  7,  0],
[32,  0,  8, 27, 30, 25,  0,  0,  5, 35,  3,  6, 23, 11,  0,  2, 26,  0,  0,  0, 21,  0, 36,  9, 17,  0, 20,  0, 14, 19, 22,  4, 34,  0, 13, 15]
]

In [1]:
(n, m) = (36, 6)
x = Var("x")
for sol in solutionsCSP(giantSudoku(s4, n, m), encoder="direct"):
    for i in range(n):
        xx = [ "%2d" % sol[x(i,j)] for j in range(n) ]
        print(" ".join(xx))
    print(status())

#### 練習問題 7.2.3: 数独の問題生成



In [1]:
import random
def uniqSol(csp):
    sols = list(solutionsCSP(csp, num=2))
    return len(sols) == 1
def generateSudoku(clues, n=9, m=3):
    positions = [ (i,j) for i in range((n+1)//2) for j in range(n) ]
    while positions:
        k = random.randrange(0, len(positions))
        (i,j) = positions.pop(k)
        (old1, old2) = (clues[i][j], clues[n-i-1][n-j-1])
        (clues[i][j], clues[n-i-1][n-j-1]) = (0, 0)
        if not uniqSol(giantSudoku(clues, n=n, m=m)):
            (clues[i][j], clues[n-i-1][n-j-1]) = (old1, old2)
    return clues

In [1]:
def initialClues(n=9, m=3):
    (row, nums) = ([], [ i+1 for i in range(n) ])
    while nums:
        k = random.randrange(0, len(nums))
        row.append(nums.pop(k))
    clues = [row] + [[0] * n] * (n-1)
    [sol] = solutionsCSP(giantSudoku(clues, n=n, m=m))
    x = Var("x")
    return [ [ sol[x(i,j)] for j in range(n) ] for i in range(n) ]

(n, m) = (9, 3)
clues = initialClues(n=n, m=m)
clues = generateSudoku(clues, n=n, m=m)
for row in clues:
    print(row)

### 7.3 覆面算



例題「覆面算1」のCSP



In [1]:
from cspsat import *
def sendMoreMoney():
    vs = [S,E,N,D,M,O,R,Y] = [ Var(v) for v in "SENDMORY" ]
    for v in vs:
        yield ["int", v, 0, 9]
    yield ["alldifferent", *vs]
    yield ["and", [">", S, 0], [">", M, 0]]
    send = ["+", ["*",1000,S], ["*",100,E], ["*",10,N], D]
    more = ["+", ["*",1000,M], ["*",100,O], ["*",10,R], E]
    money = ["+", ["*",10000,M], ["*",1000,O], ["*",100,N], ["*",10,E], Y]
    yield ["==", ["+", send, more], money]

In [1]:
solveCSP(sendMoreMoney(), stat=True)

例題「覆面算2」のCSP (copris1)



In [1]:
from cspsat import *
def copris1():
    def word(xx):
        return [ ["*",xx[len(xx)-i-1],10**i] for i in range(len(xx)) ]
    vs = [C,O,P,R,I,S,A,L,V,E] = [ Var(v) for v in "COPRISALVE" ]
    for v in vs:
        yield ["int", v, 0, 9]
    yield ["alldifferent", *vs]
    yield ["and", [">", C, 0], [">", S, 0]]
    copris = ["+", *word([C,O,P,R,I,S])]
    scala = ["+", *word([S,C,A,L,A])]
    csp = ["+", *word([C,S,P])]
    solver = ["+", *word([S,O,L,V,E,R])]
    yield ["==", ["+", copris, scala, csp], solver]

例題「覆面算2」のCSP (copris2)



In [1]:
def copris2():
    vs = [C,O,P,R,I,S,A,L,V,E] = [ Var(v) for v in "COPRISALVE" ]
    for v in vs:
        yield ["int", v, 0, 9]
    yield ["alldifferent", *vs]
    yield [">", C, 0]
    yield [">", S, 0]
    c = Var("c")
    for i in range(1,6): 
        yield ["int", c(i), 0, 2]
    yield ["==", ["+", S, A, P      ], ["+", R, ["*", 10, c(1)]]]
    yield ["==", ["+", I, L, S, c(1)], ["+", E, ["*", 10, c(2)]]]
    yield ["==", ["+", R, A, C, c(2)], ["+", V, ["*", 10, c(3)]]]
    yield ["==", ["+", P, C,    c(3)], ["+", L, ["*", 10, c(4)]]]
    yield ["==", ["+", O, S,    c(4)], ["+", O, ["*", 10, c(5)]]]
    yield ["==", ["+", C,       c(5)],       S ]

In [1]:
solveCSP(copris1(), stat=True)
solveCSP(copris2(), stat=True)

In [1]:
solveCSP(copris1(), encoder="log", stat=True)

#### 練習問題 7.3.1: 覆面算ソルバー



In [1]:
from itertools import chain, zip_longest
def alphametic(words, x=Var("x"), c=Var("c")):
    letters = set(chain(*words))
    for l in letters:
        yield ["int", x(l), 0, 9]
    yield ["alldifferent", *[ x(l) for l in letters ]]
    yield from [ [">", x(w[0]), 0] for w in words ]
    cols = list(zip_longest(*map(lambda w: reversed(w), words)))
    n = len(cols)
    for i in range(n+1):
        yield ["int", c(i), 0, len(words)-2]
    yield from [ ["==", c(0), 0], ["==", c(n), 0] ]
    for i in range(n):
        xx = [ x(l) for l in cols[i] if l ]
        yield ["==", ["+", *xx[:-1], c(i)], ["+", xx[-1], ["*", 10, c(i+1)]]]

In [1]:
words = ["COPRIS", "SCALA", "CSP", "SOLVER"]
x = Var("x")
for sol in solutionsCSP(alphametic(words)):
    zz = [ "".join([ str(sol[x(l)]) for l in w ]) for w in words ]
    print(zz)

#### 練習問題 7.3.2: NORTH/SOUTH=WEST/EAST



覆面算NORTH/SOUTH=WEST/EASTのCSP



In [1]:
from cspsat import *
def nswe():
    vs = [N, O, R, T, H, S, U, W, E, A] = [ Var(v) for v in "NORTHSUWEA" ]
    for v in vs:
        yield ["int", v, 0, 9]
    yield ["alldifferent", *vs]
    [NORTH, SOUTH, WEST, EAST] = [ Var(v) for v in ["NORTH","SOUTH","WEST","EAST"] ]
    yield ["int", NORTH, 10000, 99999]
    yield ["int", SOUTH, 10000, 99999]
    yield ["int", WEST, 1000, 9999]
    yield ["int", EAST, 1000, 9999]
    yield ["==", NORTH, ["+", ["*", 10000, N], ["*", 1000, O], ["*", 100, R], ["*", 10, T], H]]
    yield ["==", SOUTH, ["+", ["*", 10000, S], ["*", 1000, O], ["*", 100, U], ["*", 10, T], H]]
    yield ["==", WEST, ["+", ["*", 1000, W], ["*", 100, E], ["*", 10, S], T]]
    yield ["==", EAST, ["+", ["*", 1000, E], ["*", 100, A], ["*", 10, S], T]]
    x = Var()
    yield ["int", x, 10000*1000, 99999*9999]
    yield ["mulCmp", "==", NORTH, EAST, x]
    yield ["mulCmp", "==", SOUTH, WEST, x]

覆面算NORTH/SOUTH=WEST/EASTの解



In [1]:
solveCSP(nswe(), encoder="log", num=0)

### 7.4 タクシー数



In [1]:
from cspsat import *
def taxicabNumber(m, n=Var("n"), a=Var("a"), c=Var("c")):
    yield ["int", n, 0, m]
    for i in range(4):
        yield ["int", a(i), 0, m]
        yield ["int", c(i), 0, m]
        yield ["powCmp", "==", a(i), 3, c(i)]
    yield ["==", n, ["+", c(0), c(1)]]
    yield ["==", n, ["+", c(2), c(3)]]
    yield ["<", a(0), a(1)]
    yield ["<", a(2), a(3)]
    yield ["<", a(0), a(2)]

In [1]:
solveCSP(taxicabNumber(100000), encoder="log", num=0)

#### 練習問題 7.4.1: オイラー予想



In [1]:
from cspsat import *
def eulersConjecture(m=150, x=Var("x"), y=Var("y")):
    for i in range(5):
        yield ["int", x(i), 0, m]
        yield [">", x(i), 0]
        yield ["int", y(i), 0, m**5]
        yield ["powCmp", "==", x(i), 5, y(i)]
    yield ["==", ["+", y(0), y(1), y(2), y(3)], y(4)]

solveCSP(eulersConjecture(), encoder="log", command="bin/kissat")

### 7.5 コラッツ予想



コラッツ予想でのループ探索用CSP



In [1]:
from cspsat import *
def collatz(m, steps, n=Var("n")):
    for i in range(steps+1):
        yield ["int", n(i), 1, m]
    for i in range(steps):
        yield ["==", n(i+1), ["if", ["==", ["mod", n(i), 2], 0], ["div", n(i), 2], ["+", ["*", 3, n(i)], 1]]]
    yield ["or", *[ ["==", n(0), n(t)] for t in range(1,steps+1) ]]

コラッツ予想でのループ探索



In [1]:
(m, steps) = (2**300-1, 10)
n = Var("n")
for sol in solutionsCSP(collatz(m, steps), encoder="log", num=0):
    print([ sol[n(i)] for i in range(steps+1) ])

#### 練習問題 7.5.1: コラッツ予想の類似問題



コラッツ予想の類似問題でのループ探索用CSP



In [1]:
def collatzX(a, b, m, steps, n=Var("n")):
    for i in range(steps+1):
        yield ["int", n(i), 2, m]
    for i in range(steps):
        yield ["==", n(i+1), ["if", ["==", ["mod", n(i), 2], 0], ["div", n(i), 2], ["+", ["*", a, n(i)], b]]]
    yield ["or", *[ ["==", n(0), n(t)] for t in range(1,steps+1) ]]

コラッツ予想の類似問題でのループ探索



In [1]:
(m, steps) = (2**100-1, 10)
solveCSP(collatzX(3, -1, m, steps), encoder="log")
solveCSP(collatzX(5, 1, m, steps), encoder="log")

### 7.6 クイーングラフ彩色問題



クイーングラフ彩色問題のCSP (queenGraphColoring)



In [1]:
from cspsat import *
from itertools import product
def queenGraphColoring(n, k, q=Var("q")):
    def U(a): return [ (i,a-i) for i in range(n) if a-i in range(n) ]
    def D(b): return [ (i,i-b) for i in range(n) if i-b in range(n) ]
    for (i,j) in product(range(n), range(n)):
        yield ["int", q(i,j), 1, k]
    for i in range(n):
        yield ["alldifferent", *[ q(i,j) for j in range(n) ]]
    for j in range(n):
        yield ["alldifferent", *[ q(i,j) for i in range(n) ]]
    for a in range(0, 2*n-1):
        yield ["alldifferent", *[ q(i,j) for (i,j) in U(a) ]]
    for b in range(-n+1, n):
        yield ["alldifferent", *[ q(i,j) for (i,j) in D(b) ]]
    for j in range(n):
        yield ["==", q(0,j), j+1]

In [1]:
n = 5
q = Var("q")
for sol in solutionsCSP(queenGraphColoring(n, n), encoder="direct", num=0):
    for i in range(n):
        print([ sol[q(i,j)] for j in range(n) ])
    print()

#### 練習問題 7.6.1: ラテン方陣とオイラー方陣



In [1]:
def latinSquare(n, x=Var("x")):
    for i in range(n):
        for j in range(n):
            yield ["int", x(i,j), 1, n]
    for i in range(n):
        yield ["alldifferent", *[ x(i,j) for j in range(n) ]]
    for j in range(n):
        yield ["alldifferent", *[ x(i,j) for i in range(n) ]]
    for j in range(n):
        yield ["==", x(0,j), j+1]

def eulerSquare(n, x=Var("x"), y=Var("y")):
    yield from latinSquare(n, x)
    yield from latinSquare(n, y)
    for i in range(1,n):
        yield ["==", x(i,0), i+1]
    zz = [ ["+", ["*", n, x(i,j)], y(i,j), -n] for i in range(n) for j in range(n) ]
    yield ["alldifferent", *zz]

In [1]:
n = 4
(x, y) = (Var("x"), Var("y"))
for sol in solutionsCSP(eulerSquare(n, x, y)):
    for i in range(n):
        print([ (sol[x(i,j)],sol[y(i,j)]) for j in range(n) ])

#### 練習問題 7.6.2: 優美なグラフ



優美なラベル付けのCSP



In [1]:
from cspsat import *
def gracefulLabeling(vertices, edges, x=Var("x"), y=Var("y")):
    (n, m) = (len(vertices), len(edges))
    for v in vertices:
        yield ["int", x(v), 0, m]
    yield ["alldifferent", *[ x(v) for v in vertices ]]
    for (u,v) in edges:
        yield ["int", y(u,v), 1, m]
        yield ["==", y(u,v), ["abs", ["-", x(u), x(v)]]]
    yield ["alldifferent", *[ y(u,v) for (u,v) in edges ]]

優美なラベル付の実行例



In [1]:
from cspsat.examples.graph import completeGraph, wheelGraph
(vertices, edges) = completeGraph(4)
solveCSP(gracefulLabeling(vertices, edges))
(vertices, edges) = wheelGraph(5)
solveCSP(gracefulLabeling(vertices, edges))

#### 練習問題 7.6.3: nクイーン再び



In [1]:
from cspsat import *
def nqueens(n, q=Var("q")):
    for i in range(n):
        yield ["int", q(i), 0, n-1]
    yield ["alldifferent", *[ q(i) for i in range(n) ]]
    yield ["alldifferent", *[ ["+", q(i), i] for i in range(n) ]]
    yield ["alldifferent", *[ ["-", q(i), i] for i in range(n) ]]

solveCSP(nqueens(8))

#### 練習問題 7.6.4: 総音程音列



In [1]:
from cspsat import *
def allIntervalSeries(n, x=Var("x"), d=Var("d")):
    for i in range(n):
        yield ["int", x(i), 0, n-1]
    yield ["alldifferent", *[ x(i) for i in range(n) ]]
    for i in range(n-1):
        yield ["int", d(i), 1, n-1]
        yield ["==", d(i), ["mod", ["-", x(i+1), x(i)], n]]
    yield ["alldifferent", *[ d(i) for i in range(n-1) ]]

In [1]:
x = Var("x")
for sol in solutionsCSP(allIntervalSeries(12)):
    name = ["A","A#","B","C","C#","D","D#","E","F","F#","G","G#"]
    print([ name[sol[x(i)]] for i in range(12) ])

### 7.7 クイーン支配問題



クイーン支配問題のCSP (queenDomination1)



In [1]:
from cspsat import *
from itertools import product
def queenDomination1(n, s, q=Bool("q")):
    def attacked(i, j, k, l):
        return i == k or j == l or i+j == k+l or i-j == k-l
    for (i,j) in product(range(n), range(n)):
        qq = [ q(k,l) for k in range(n) for l in range(n) if attacked(i,j,k,l) ]
        yield ["or", *qq]
    yield ["eqK", [ q(i,j) for i in range(n) for j in range(n) ], s]

8次のクイーングラフは5個のクイーンで支配できる



In [1]:
solveCSP(queenDomination1(8, 5), positiveOnly=True)

8次のクイーングラフは4個のクイーンでは支配できない



In [1]:
solveCSP(queenDomination1(8, 4), positiveOnly=True)

In [1]:
solveCSP(queenDomination1(11, 5), positiveOnly=True, stat=True)

クイーン支配問題のCSP (queenDomination)



In [1]:
def queenDomination(n, s, q=Bool("q")):
    def U(a): return [ (i,a-i) for i in range(n) if a-i in range(n) ]
    def D(b): return [ (i,i-b) for i in range(n) if i-b in range(n) ]
    (r, c, u, d) = (Bool(), Bool(), Bool(), Bool())
    for i in range(n):
        yield ["equ", r(i), ["or", *[ q(i,j) for j in range(n)]]]
    for j in range(n):
        yield ["equ", c(j), ["or", *[ q(i,j) for i in range(n)]]]
    for a in range(0, 2*n-1):
        yield ["equ", u(a), ["or", *[ q(i,j) for (i,j) in U(a)]]]
    for b in range(-n+1, n):
        yield ["equ", d(b), ["or", *[ q(i,j) for (i,j) in D(b)]]]
    for (i,j) in product(range(n), range(n)):
        yield ["or", r(i), c(j), u(i+j), d(i-j)]
    yield ["==", ["+", *[ q(i,j) for i in range(n) for j in range(n) ]], s]
    yield ["<=", ["+", *[ r(i) for i in range(n) ]], s]
    yield ["<=", ["+", *[ c(j) for j in range(n) ]], s]
    yield ["<=", ["+", *[ u(a) for a in range(0, 2*n-1) ]], s]
    yield ["<=", ["+", *[ d(b) for b in range(-n+1, n) ]], s]

In [1]:
solveCSP(queenDomination(13, 7), positiveOnly=True)

In [1]:
solveCSP(queenDomination(14, 8), positiveOnly=True, command="bin/kissat")

クイーン支配問題のCOP (queenDominationOpt)



In [1]:
def queenDominationOpt(n, s=Var("s"), q=Bool("q")):
    def U(a): return [ (i,a-i) for i in range(n) if a-i in range(n) ]
    def D(b): return [ (i,i-b) for i in range(n) if i-b in range(n) ]
    (r, c, u, d) = (Bool(), Bool(), Bool(), Bool())
    for i in range(n):
        yield ["equ", r(i), ["or", *[ q(i,j) for j in range(n)]]]
    for j in range(n):
        yield ["equ", c(j), ["or", *[ q(i,j) for i in range(n)]]]
    for a in range(0, 2*n-1):
        yield ["equ", u(a), ["or", *[ q(i,j) for (i,j) in U(a)]]]
    for b in range(-n+1, n):
        yield ["equ", d(b), ["or", *[ q(i,j) for (i,j) in D(b)]]]
    for (i,j) in product(range(n), range(n)):
        yield ["or", r(i), c(j), u(i+j), d(i-j)]
    yield ["int", s, 1, n]
    yield ["==", ["+", *[ q(i,j) for i in range(n) for j in range(n) ]], s]
    yield ["<=", ["+", *[ r(i) for i in range(n) ]], s]
    yield ["<=", ["+", *[ c(j) for j in range(n) ]], s]
    yield ["<=", ["+", *[ u(a) for a in range(0, 2*n-1) ]], s]
    yield ["<=", ["+", *[ d(b) for b in range(-n+1, n) ]], s]
    yield ["minimize", s]

In [1]:
solveCSP(queenDominationOpt(11), positiveOnly=True, stat=True)

#### 練習問題 7.7.1: クイーングラフの独立支配数



クイーングラフの独立支配集合のCSP



In [1]:
def qidp(n, s, q=Bool("q"), r=Bool("r"), c=Bool("c"), u=Bool("u"), d=Bool("d")):
    for i in range(n):
        yield ["==", r(i), ["+", *[ q(i,j) for j in range(n)]]]
    for j in range(n):
        yield ["==", c(j), ["+", *[ q(i,j) for i in range(n)]]]
    for a in range(0, 2*n-1):
        qq = [ q(i,a-i) for i in range(n) if a-i in range(n) ]
        yield ["==", u(a), ["+", *qq]]
    for b in range(-n+1, n):
        qq = [ q(i,i-b) for i in range(n) if i-b in range(n) ]
        yield ["==", d(b), ["+", *qq]]
    for (i,j) in product(range(n), range(n)):
        yield ["or", r(i), c(j), u(i+j), d(i-j)]
    yield ["eqK", [ r(i) for i in range(n) ], s]
    yield ["eqK", [ c(j) for j in range(n) ], s]
    yield ["eqK", [ u(a) for a in range(0, 2*n-1) ], s]
    yield ["eqK", [ d(b) for b in range(-n+1, n) ], s]

In [1]:
q = Bool("q")
n = 8
for sol in solutionsCSP(qidp(8, 5)):
    for i in range(n):
        print(" ".join([ "Q" if sol[q(i,j)] else "." for j in range(n) ]))

In [1]:
solveCSP(qidp(8, 4), positiveOnly=True)

#### 練習問題 7.7.2: 支配集合



支配集合のCSP



In [1]:
from cspsat import *
def dominatingSet(vertices, edges, k=None, x=Bool("x")):
    def adj(v):
        return [ e[1-i] for e in edges for i in [0,1] if e[i] == v ]
    for v in vertices:
        yield ["or", x(v), *[ x(u) for u in adj(v) ]]
    if k:
        yield ["leK", [ x(v) for v in vertices ], k]

In [1]:
from cspsat.examples.graph import superQueenGraph
(vertices, edges) = superQueenGraph(8)
solveCSP(dominatingSet(vertices, edges, 3), positiveOnly=True)
solveCSP(dominatingSet(vertices, edges, 2), positiveOnly=True)

#### 練習問題 7.7.3: 独立集合



独立集合のCSP



In [1]:
from cspsat import *
def independentSet(vertices, edges, k=None, x=Bool("x")):
    for (u,v) in edges:
        yield ["leK", [x(u), x(v)], 1]
    if k:
        yield ["geK", [ x(v) for v in vertices ], k]

In [1]:
from cspsat.examples.graph import superQueenGraph
for n in range(1, 11):
    print(f"# {n}")
    (vertices, edges) = superQueenGraph(n)
    solveCSP(independentSet(vertices, edges, n), positiveOnly=True)

### 7.8 正方形の詰込み



正方形詰込み問題のCSP (squarePacking)



In [1]:
from cspsat import *
from itertools import combinations
def squarePacking(s, size, x=Var("x"), y=Var("y")):
    n = len(s)
    for i in range(n):
        yield ["int", x(i), 0, size-s[i]]
        yield ["int", y(i), 0, size-s[i]]
    for (i,j) in combinations(range(n), 2):
        yield ["or",
                      ["<=", ["+", x(i), s[i]], x(j)],
                      ["<=", ["+", x(j), s[j]], x(i)],
                      ["<=", ["+", y(i), s[i]], y(j)],
                      ["<=", ["+", y(j), s[j]], y(i)]
        ]
    yield ["<=", x(0), ["-", size, ["+", x(0), s[0]]]]
    yield ["<=", y(0), ["-", size, ["+", y(0), s[0]]]]
    yield [">=", x(0), y(0)]

例題「完全正方形分割」の解



In [1]:
s = [2,4,6,7,8,9,11,15,16,17,18,19,24,25,27,29,33,35,37,42,50]
size = 112
solveCSP(squarePacking(s, size))

In [1]:
def showSquarePacking(sol, s, size, x=Var("x"), y=Var("y")):
    p = [ [ "." for b in range(size) ] for a in range(size) ]
    for i in range(len(s)):
        (a, b) = (sol[y(i)], sol[x(i)])
        for (da,db) in product(range(s[i]), range(s[i])):
            p[a+da][b+db] = chr(ord("A")+i) if i < 26 else chr(ord("a")+i-26)
    for a in reversed(range(size)):
        print("".join(p[a]))

s = [2,4,6,7,8,9,11,15,16,17,18,19,24,25,27,29,33,35,37,42,50]
size = 112
for sol in solutionsCSP(squarePacking(s, size)):
    showSquarePacking(sol, s, size)

例題「正方形詰込み問題」の解



In [1]:
n = 10
s = range(1, n+1)
solveCSP(squarePacking(s, 20))
solveCSP(squarePacking(s, 21))

In [1]:
n = 10
s = range(1, n+1)
size = 21
for sol in solutionsCSP(squarePacking(s, size)):
    showSquarePacking(sol, s, size)

例題「正方形詰込み問題」のCOP (squarePackingOpt)



In [1]:
def squarePackingOpt(n, x=Var("x"), y=Var("y"), size=Var("size")):
    sizeLb = math.ceil(math.sqrt(n*(n+1)*(2*n+1)/6))
    sizeUb = sizeLb + 2
    yield from squarePacking(range(1,n+1), sizeUb)
    yield ["int", size, sizeLb, sizeUb]
    for i in range(n):
        yield ["<=", ["+", x(i), i+1], size]
        yield ["<=", ["+", y(i), i+1], size]
    yield ["minimize", size]

In [1]:
solveCSP(squarePackingOpt(25))

In [1]:
solveCSP(squarePackingOpt(30), command="bin/kissat")

#### 練習問題 7.8.1: ペントミノ



In [1]:
from cspsat.examples.polyomino import *
(m, n) = (6, 10)
for sol in solutionsCSP(polyomino(m, n, pentominos), encoder="direct"):
    showPolyomino(sol, m, n, pentominos)

#### 練習問題 7.8.2: Yペントミノの長方形敷き詰め



In [1]:
from cspsat.examples.polyomino import *
(m, n, k) = (5, 10, 10)
yShape = [ shape for (n,shape) in pentominos if n == "Y" ][0]
ominos = [ (chr(ord("A")+i),yShape) for i in range(k) ]
for sol in solutionsCSP(polyomino(m, n, ominos), encoder="direct"):
    showPolyomino(sol, m, n, ominos)

### 7.9 四角フリー行列



例題「四角フリー行列」のCSP (quadFree)



In [1]:
from cspsat import *
from itertools import combinations
def quadFree(n, k, x=Bool("x")):
    for (i,i1) in combinations(range(n), 2):
        for (j,j1) in combinations(range(n), 2):
            yield ["leK", [x(i,j), x(i,j1), x(i1,j), x(i1,j1)], 3]
    xx = [ x(i,j) for i in range(n) for j in range(n) ]
    yield ["geK", xx, k]

In [1]:
solveCSP(quadFree(4, 9), positiveOnly=True)

In [1]:
solveCSP(quadFree(4, 10), positiveOnly=True)

#### 練習問題 7.9.1: 四角フリー行列の対称性除去



In [1]:
from cspsat import *
def quadFreeSb(n, k, x=Bool("x")):
    yield from quadFree(n, k)
    for i in range(n-1):
        yield ["lexCmp", "<=", [ x(i,j) for j in range(n) ], [ x(i+1,j) for j in range(n) ]]
    for j in range(n-1):
        yield ["lexCmp", "<=", [ x(i,j) for i in range(n) ], [ x(i,j+1) for i in range(n) ]]

In [1]:
(n, k) = (4, 9)
x = Bool("x")
for sol in solutionsCSP(quadFreeSb(n, k), num=0, command="bin/kissat"):
    for i in range(n):
        print(" ".join([ "#" if sol[x(i,j)] else "." for j in range(n) ]))
    print()

### 7.10 カークマンの女学生問題



分割可能なBIBDのCSP (rbibd1，対称性除去なし)



In [1]:
from cspsat import *
from itertools import combinations, product
def rbibd1(v, b, r, k, x=Bool("x")):
    s = b//r
    for (i,d) in product(range(v), range(r)):
        yield ["eqK", [ x(i,s*d+j) for j in range(s) ], 1]
    for j in range(b):
        yield ["eqK", [ x(i,j) for i in range(v) ], k]
    for (i, i1) in combinations(range(v), 2):
        for (j, j1) in combinations(range(b), 2):
            yield ["leK", [x(i,j), x(i,j1), x(i1,j), x(i1,j1)], 3]

In [1]:
csp = rbibd1(15, 35, 7, 3)
x = Bool("x")
for sol in solutionsCSP(csp):
    for i in range(15):
        ss = [ "#" if sol[x(i,j)] else "." for j in range(35) ]
        print(" ".join(ss))

分割可能BIBDのCSP (rbibd1sb，対称性除去あり)



In [1]:
def rbibd1sb(v, b, r, k, x=Bool("x")):
    yield from rbibd1(v, b, r, k, x=x)
    s = b//r
    for j in range(s):
        yield ["and", *[ x(k*j+i,j) for i in range(k) ]]
    for i in range(k):
        yield ["and", *[ x(i,s*d+i) for d in range(1,r) ]]

In [1]:
csp = rbibd1sb(15, 35, 7, 3)
x = Bool("x")
for sol in solutionsCSP(csp):
    for i in range(15):
        ss = [ "#" if sol[x(i,j)] else "." for j in range(35) ]
        print(" ".join(ss))

#### 練習問題 7.10.2: カークマンの女学生問題での対称性除去



In [1]:
solveCSP(rbibd1(15, 35, 7, 3), positiveOnly=True, stat=True)
solveCSP(rbibd1sb(15, 35, 7, 3), positiveOnly=True, stat=True)

#### 練習問題 7.10.3: 麻雀の組合せ



In [1]:
csp = rbibd1sb(16, 20, 5, 4)
x = Bool("x")
for sol in solutionsCSP(csp):
    for i in range(16):
        ss = [ "#" if sol[x(i,j)] else "." for j in range(20) ]
        print(" ".join(ss))

### 7.11 回転式定規



巡回差集合のCSP (cyclicDiffSet1)



In [1]:
from cspsat import *
def cyclicDiffSet1(v, k, x=Bool("x")):
    yield ["eqK", [ x(i) for i in range(v) ], k]
    for d in range(1,v//2+1):
        yield ["or", *[ ["and", x(i), x((i+d)%v)] for i in range(v) ]]
    yield x(0)
    yield x(1)

In [1]:
solveCSP(cyclicDiffSet1(13, 4), positiveOnly=True)

In [1]:
k = 5
solveCSP(cyclicDiffSet1(k*(k-1)+1, k), positiveOnly=True)

In [1]:
k = 7
solveCSP(cyclicDiffSet1(k*(k-1)+1, k), positiveOnly=True)

#### 練習問題 7.11.2: ゴロム定規



ゴロム定規のCSP



In [1]:
from cspsat import *
from itertools import combinations
def golombRuler(m, n, x=Bool("x")):
    yield ["eqK", [ x(i) for i in range(n+1) ], m]
    yield ["and", x(0), x(n)]
    for (i,j) in combinations(range(n+1), 2):
        for d in range(1,n):
            if i+d <= j and j+d <= n:
                yield ["or", ~x(i), ~x(i+d), ~x(j), ~x(j+d)]

In [1]:
for (m,n) in [(11,72), (12,85)]:
    solveCSP(golombRuler(m, n), positiveOnly=True, stat=True)

#### 練習問題 7.11.3: スパース定規



スパース定規のCSP



In [1]:
from cspsat import *
from itertools import combinations
def sparsebRuler(m, n, x=Bool("x")):
    yield ["eqK", [ x(i) for i in range(n+1) ], m]
    yield ["and", x(0), x(n)]
    for d in range(1,n+1):
        yield ["or", *[ ["and", x(i), x(i+d)] for i in range(n) if i+d <= n ]]
    yield x(1)

In [1]:
(m, n) = (13, 58)
solveCSP(sparsebRuler(m, n), positiveOnly=True, stat=True)

### 7.12 ノノグラム



ノノグラムのCSP (nonogram)



In [1]:
from cspsat import *
from itertools import product
def nonogram(rows, cols, x=Bool("x")):
    (m, n) = (len(rows), len(cols))
    (h, v) = (Var(), Var())
    for i in range(m):
        clues = rows[i]
        for k in range(len(clues)):
            yield ["int", h(i,k), 0, n-clues[k]]
        for k in range(len(clues)-1):
            yield ["<", ["+", h(i,k), clues[k]], h(i,k+1)]
    for j in range(n):
        clues = cols[j]
        for k in range(len(clues)):
            yield ["int", v(j,k), 0, m-clues[k]]
        for k in range(len(clues)-1):
            yield ["<", ["+", v(j,k), clues[k]], v(j,k+1)]
    for (i,j) in product(range(m), range(n)):
        (clues1, clues2) = (rows[i], cols[j])
        cs1 = [ ["and", ["<=", h(i,k), j], ["<", j, ["+", h(i,k), clues1[k]]]] for k in range(len(clues1)) ]
        yield ["equ", x(i,j), ["or", *cs1]]
        cs2 = [ ["and", ["<=", v(j,k), i], ["<", i, ["+", v(j,k), clues2[k]]]] for k in range(len(clues2)) ]
        yield ["equ", x(i,j), ["or", *cs2]]

In [1]:
rows = [[3], [1], [1,3], [1], [1,3], [1], [1]]
cols = [[3], [1], [1,3], [1], [1,3], [1], [1]]
x = Bool("x")
for sol in solutionsCSP(nonogram(rows, cols)):
    for i in range(len(rows)):
        print([ sol[x(i,j)] for j in range(len(cols)) ])

非決定有限オートマトン (NFA)のCSP (nfa)



In [1]:
from cspsat import *
from itertools import product
def nfa(sigma, delta, fins, xx):
    (m, n) = (len(delta), len(xx))
    s = Var()
    for i in range(n+1):
        yield ["int", s(i), 0, m-1]
    yield ["==", s(0), 0]
    yield ["or", *[ ["==", s(n), q] for q in fins ]]
    for (i,q,a) in product(range(n), range(m), sigma):
        c1 = ["and", ["==", s(i), q], ["==", xx[i], a]]
        c2 = [ ["==", s(i+1), q1] for q1 in delta[q].get(a, []) ]
        yield ["imp", c1, ["or", *c2]]

In [1]:
def pattern13(x=Var("x")):
    sigma = [0, 1]
    delta = [ {0:{0},1:{1}}, {0:{2}}, {0:{2},1:{3}}, {1:{4}}, {1:{5}}, {0:{5}} ]
    fins = [5]
    n = 7
    for i in range(n):
        yield ["int", x(i), 0, 1]
    yield from nfa(sigma, delta, fins, [ x(i) for i in range(n) ])

solveCSP(pattern13(), num=0)

In [1]:
def nfaByPattern(pattern, xx):
    if pattern == [1]:
        delta = [ {0:{0},1:{1}}, {0:{1}} ]
        yield from nfa([0,1], delta, [1], xx)
    elif pattern == [3]:
        delta = [ {0:{0},1:{1}}, {1:{2}}, {1:{3}}, {0:{3}} ]
        yield from nfa([0,1], delta, [3], xx)
    elif pattern == [1,3]:
        delta = [ {0:{0},1:{1}}, {0:{2}}, {0:{2},1:{3}}, {1:{4}}, {1:{5}}, {0:{5}} ]
        yield from nfa([0,1], delta, [5], xx)
    else:
        raise Exception(f"Unknown NFA pattern: {pattern}")

例題「ノノグラム」のCSP (nonogramNFA，非決定性有限オートマトンを利用)



In [1]:
def nonogramNFA(rows, cols, x=Var("x")):
    (m, n) = (len(rows), len(cols))
    for (i,j) in product(range(m), range(n)):
        yield ["int", x(i,j), 0, 1]
    for i in range(m):
        yield from nfaByPattern(rows[i], [ x(i,j) for j in range(n) ])
    for j in range(n):
        yield from nfaByPattern(cols[j], [ x(i,j) for i in range(n) ])

rows = cols = [[3], [1], [1,3], [1], [1,3], [1], [1]]
solveCSP(nonogramNFA(rows, cols), num=0)

#### 練習問題 7.12.1: 100×100の問題



In [1]:
rows = [[2],[2],[2],[2],[12],[2],[2],[2],[2],[2],[2],[2],[2],[2],[2],[2],[14],[14],[1,2],[1,2],[1,2],[1,2],[11,2],[1,2],[1,2],[1,2],[1,2],[1,5],[1,5],[1,5],[1,5],[1,5],[1,5],[1,24],[32],[26],[26],[26],[3,8,7,2],[3,7,6,2],[3,7,6,2],[3,7,6,2],[3,8,7,2],[26],[34],[34],[34],[34],[34],[24],[5,3,5],[3,5,3,5],[2,2,24],[1,1,24],[2,29],[5,24,2],[3,26,2],[41,3],[43,2],[42,3],[41,2],[41,2],[4,2,2,2,2,5],[42],[42],[42],[64],[86],[83],[65],[75],[71],[60,10],[6,53,10],[6,25,24,10],[19,17,3,44],[30,12,40],[12,19,22],[9,24,16],[4,20,8,11],[5,24,6,2,3],[38,6,2,6],[45,8,8],[52,3],[51,1],[51,2],[50,1],[50,1],[50,1],[34,9,2],[31,6,7,7,1],[29,10,6,3,5,1],[27,8,5,5,3,1],[25,5,5,4,10,5,2],[23,3,11,2,3,3,5,1],[20,3,16,2,8,11,1],[18,2,8,4,1,9,3,7,2],[15,4,8,5,4,12,5,9,1],[11,5,7,8,2,14,5,11,1],[7,7,6,13,9,15,15]]
cols = [[2],[4],[8],[2,13],[2,17],[2,18],[2,19],[2,19],[2,19],[1,2,19],[2,2,19],[2,2,19],[2,2,19],[2,2,19],[2,2,19],[2,2,18],[2,26,1],[2,9,17,1],[2,10,17,1],[2,10,16,2],[2,9,17,2],[2,9,17,2],[2,3,3,16,3],[13,16,2],[13,15,1],[12,16,2,1],[12,15,2,1],[12,15,1,2],[11,15,2,3],[11,15,1,3],[11,14,2,4],[1,3,20,14,1,3],[1,1,2,23,13,1,4],[1,1,1,21,14,2,4],[1,1,5,2,22,13,3,3,1],[1,1,5,4,5,13,13,2,3,1],[1,1,5,2,5,13,12,2,3,1],[31,1,6,13,12,3,3,2],[1,15,1,6,13,12,3,2,2],[1,29,13,12,3,2,3],[1,5,32,13,3,2,3],[1,42,13,2,3,3],[1,29,12,13,2,3,3],[29,12,14,1,3,3],[2,17,10,12,4,8,3,2],[1,2,17,10,12,3,8,4,1],[1,2,17,10,12,3,2,9,4,1],[1,2,17,23,3,2,10,3],[1,2,6,8,22,3,2,11,3],[1,2,11,19,11,4,2,11,3],[38,19,11,4,2,13,1],[38,19,11,25],[1,2,11,7,10,12,1,1,3],[1,2,12,8,10,12,1,1,5],[1,2,17,23,1,1,7],[1,2,17,24,1,1,7],[1,2,17,10,13,1,1,8],[2,17,10,13,1,1,3,5],[29,13,4,3,4],[29,14,2,4,4],[5,19,14,1,2,4],[29,14,1,1,3],[44,2,2,2],[5,20,2,3,1],[5,5,14,2,2,2],[5,5,15,3,2,1],[5,5,15,3,3,1],[5,5,15,6,1],[5,15,3,3],[21,3,3],[21,3,3],[22,2,2],[2,17,2,2],[2,2,13,3,1],[1,2,13,2,1],[2,2,13,2,1],[2,1,6,5,4],[1,2,14,4],[1,14,4],[14,2],[14,2],[14,3],[14,2],[14,2],[15,2],[15,2],[15,2],[5,2,1,1],[2,1,2,1,1],[2,1,2,1,1],[2,1,2,1,1],[2,1,2,2,1],[2,1,2,2,1],[1,2,1,1],[1,2,1,1],[2,1,4],[2,1,4],[2,2,5],[2,2,5],[3]]
x = Bool("x")
for sol in solutionsCSP(nonogram(rows, cols)):
    for i in range(len(rows)):
        print("".join([ "#" if sol[x(i,j)] else "." for j in range(len(cols)) ]))
print(status())

### 7.13 ナイトツアー



ナイトグラフを返す関数 (knightGraph)



In [1]:
from itertools import combinations
def knightGraph(n):
    vertices = [ (i,j) for i in range(n) for j in range(n) ]
    edges = []
    for (i,j) in vertices:
        for (k,l) in [(i+1,j-2), (i+1,j+2), (i+2,j-1), (i+2,j+1)]:
            if k in range(n) and l in range(n):
                edges.append(((i,j), (k,l)))
    return (vertices, edges)

ハミルトン閉路のCSP (hamiltonianCycle)



In [1]:
from cspsat import *
def hamiltonianCycle(vertices, edges, x=Var("x"), a=Bool("a")):
    def adj(v):
        return [ e[1-i] for e in edges for i in [0,1] if e[i] == v ]
    for (u,v) in edges:
        yield ["<=", ["+", a(u,v), a(v,u)], 1]
    for v in vertices:
        yield ["==", ["+", *[ a(v,u) for u in adj(v) ]], 1]
        yield ["==", ["+", *[ a(u,v) for u in adj(v) ]], 1]
    s = vertices[0]
    yield ["int", x(s), 0, 0]
    for v in vertices[1:]:
        yield ["int", x(v), 1, len(vertices)-1]
    for u in vertices:
        for v in [ v for v in adj(u) if v != s ]:
            yield ["imp", a(u,v), ["==", x(v), ["+", x(u), 1]]]

例題「ナイトツアー」の解



In [1]:
n = 8
(vertices, edges) = knightGraph(n)
x = Var("x")
for sol in solutionsCSP(hamiltonianCycle(vertices, edges)):
    for i in range(n):
        ss = [ sol[x((i,j))]+1 for j in range(n) ]
        print("".join([ ("%4d" % s) for s in ss ]))

#### 練習問題 7.13.2: 点対称なナイトツアー



点対称なナイトツアーのCSP



In [1]:
from cspsat.examples.graph import knightGraph, hamiltonianCycle
def symmetricKnightTour(n, x=Var("x"), a=Bool("a")):
    (vertices, edges) = knightGraph(n)
    yield from hamiltonianCycle(vertices, edges, x=x, a=a)
    for (u,v) in edges:
        ((i1,j1), (i2,j2)) = (u, v)
        (u2, v2) = ((n-1-i1,n-1-j1), (n-1-i2,n-1-j2))
        if (u,v) < (u2,v2):
            yield ["equ", a(u,v), a(u2,v2)]
            yield ["equ", a(v,u), a(v2,u2)]

点対称なナイトツアーの解



In [1]:
n = 8
x = Var("x")
for sol in solutionsCSP(symmetricKnightTour(n)):
    for i in range(n):
        ss = [ sol[x((i,j))]+1 for j in range(n) ]
        print("".join([ ("%4d" % s) for s in ss ]))

#### 練習問題 7.13.3: ナイトパス



In [1]:
from cspsat import *
from cspsat.examples.graph import knightGraph, hamiltonianCycle
def hamiltonianPath(vertices, edges, x=Var("x"), a=Bool("a")):
    s = None
    edges = edges + [ (s,v) for v in vertices ]
    vertices = [s] + vertices
    yield from hamiltonianCycle(vertices, edges, x=x, a=a)

In [1]:
n = 8
(vertices, edges) = knightGraph(n)
x = Var("x")
for sol in solutionsCSP(hamiltonianPath(vertices, edges, x=x)):
    for i in range(n):
        ss = [ sol[x((i,j))] for j in range(n) ]
        print("".join([ ("%4d" % s) for s in ss ]))

#### 練習問題 7.13.4: 交差しない最長ナイトパス



In [1]:
from cspsat import *
from cspsat.examples.graph import knightGraph, singlePath
from itertools import combinations
def uncrossedKnightPath(n, minLen, maxLen=None, e=Bool("e"), x=Var("x")):
    def side(w, line):
        (x,y) = w
        ((x1,y1),(x2,y2)) = line
        s = (x1-x2)*(y-y1) + (y1-y2)*(x1-x)
        return -1 if s < 0 else 0 if s == 0 else 1
    def crossing(e1, e2):
        return side(e1[0], e2)*side(e1[1], e2) < 0 and side(e2[0], e1)*side(e2[1], e1) < 0
    (vertices, edges) = knightGraph(n)
    yield from singlePath(vertices, edges, minLen=minLen, maxLen=maxLen, e=e, x=x)
    for (e1,e2) in combinations(edges, 2):
        if crossing(e1, e2):
            ((u1,v1), (u2,v2)) = (e1, e2)
            yield ["<=", ["+", e(u1,v1), e(u2,v2)], 1]

In [1]:
(n, l) = (8, 35)
x = Var("x")
for sol in solutionsCSP(uncrossedKnightPath(n, l, l), command="bin/kissat"):
    for i in range(n):
        ss = [ sol[x((i,j))] for j in range(n) ]
        print(" ".join([ "%2d" % s if s > 0 else " ." for s in ss ]))

#### 練習問題 7.13.5: 交差しない最長ナイトツアー



In [1]:
from cspsat import *
from cspsat.examples.graph import knightGraph, singleCycle
from itertools import combinations
def uncrossedKnightTour(n, minLen, maxLen=None, e=Bool("e"), x=Var("x")):
    def side(w, line):
        (x,y) = w
        ((x1,y1),(x2,y2)) = line
        s = (x1-x2)*(y-y1) + (y1-y2)*(x1-x)
        return -1 if s < 0 else 0 if s == 0 else 1
    def crossing(e1, e2):
        return side(e1[0], e2)*side(e1[1], e2) < 0 and side(e2[0], e1)*side(e2[1], e1) < 0
    (vertices, edges) = knightGraph(n)
    yield from singleCycle(vertices, edges, minLen=minLen, maxLen=maxLen, e=e, x=x)
    for (e1,e2) in combinations(edges, 2):
        if crossing(e1, e2):
            ((u1,v1), (u2,v2)) = (e1, e2)
            yield ["<=", ["+", e(u1,v1), e(u2,v2)], 1]

In [1]:
(n, l) = (8, 32)
x = Var("x")
for sol in solutionsCSP(uncrossedKnightTour(n, l, l), command="bin/kissat"):
    for i in range(n):
        ss = [ sol[x((i,j))] for j in range(n) ]
        print(" ".join([ "%2d" % s if s > 0 else " ." for s in ss ]))

### 7.14 碁石拾い



In [1]:
def goishiGraph(board):
    (m, n) = (len(board), len(board[0]))
    vertices = [ (i,j) for i in range(m) for j in range(n) if board[i][j] != "." ]
    edges = []
    for (i,j) in vertices:
        edges.extend([ ((i,j),(k,j)) for k in range(i+1,m) if (k,j) in vertices ])
        edges.extend([ ((i,j),(i,l)) for l in range(j+1,n) if (i,l) in vertices ])
    return (vertices, edges)

In [1]:
board = [
    ".#####",
    "..#...",
    ".####.",
    "#.#.#.",
    "..###.",
]
(vertices, edges) = goishiGraph(board)
print(vertices)
print(edges)

例題「碁石拾い」の不完全な解



In [1]:
from cspsat import *
from cspsat.examples.graph import hamiltonianPath
(vertices, edges) = goishiGraph(board)
(m, n) = (len(board), len(board[0]))
x = Var("x")
for sol in solutionsCSP(hamiltonianPath(vertices, edges, x=x)):
    for i in range(m):
        ss = [ sol[x((i,j))] if board[i][j] != "." else 0 for j in range(n) ]
        print(" ".join([ "%2d" % s if s > 0 else " ." for s in ss ]))

In [1]:
def between(w, u, v):
    if w[0] == u[0] == v[0]:
        return min(u[1],v[1]) < w[1] < max(u[1],v[1])
    if w[1] == u[1] == v[1]:
        return min(u[0],v[0]) < w[0] < max(u[0],v[0])
    return False

例題「碁石拾い」のCSP



In [1]:
from itertools import permutations
def goishiHiroi(board, a=Bool("a"), x=Var("x")):
    (vertices, edges) = goishiGraph(board)
    yield from hamiltonianPath(vertices, edges, x=x, a=a)
    for v in vertices:
        us1 = [ u for u in vertices if u[0] == v[0] and u[1] < v[1] ]
        us2 = [ u for u in vertices if u[0] == v[0] and u[1] > v[1] ]
        us3 = [ u for u in vertices if u[0] < v[0] and u[1] == v[1] ]
        us4 = [ u for u in vertices if u[0] > v[0] and u[1] == v[1] ]
        for us in [us1,us2,us3,us4]:
            for [u1,u2] in permutations(us, 2):
                yield ["or", ~a(u1,v), ~a(v,u2)]
    for (u,v) in edges:
        ws = [ w for w in vertices if between(w, u, v) ]
        cs = [ ["and", ["<", x(w), x(u)], ["<", x(w), x(v)]] for w in ws ]
        yield ["imp", ["or", a(u,v), a(v,u)], ["and", *cs]]

In [1]:
(m, n) = (len(board), len(board[0]))
x = Var("x")
for sol in solutionsCSP(goishiHiroi(board)):
    for i in range(m):
        ss = [ sol[x((i,j))] if board[i][j] != "." else 0 for j in range(n) ]
        print(" ".join([ "%2d" % s if s > 0 else " ." for s in ss ]))

#### 練習問題 7.14.1: 銀のハミルトン路



In [1]:
from cspsat import *
def ginDigraph(m, n):
    vertices = [ (i,j) for i in range(m) for j in range(n) ]
    arcs = []
    for (i,j) in vertices:
        for (k,l) in [(i-1,j-1), (i-1,j), (i-1,j+1), (i+1,j-1), (i+1,j+1)]:
            if k in range(m) and l in range(n):
                arcs.append(((i,j),(k,l)))
    return (vertices, arcs)

In [1]:
from cspsat.examples.graph import *
(m, n) = (4, 4)
x = Var("x")
(vertices, arcs) = ginDigraph(m, n)
for sol in solutionsCSP(digraphHamiltonianPath(vertices, arcs, x=x)):
    for i in range(n):
        ss = [ "%2d" % sol[x((i,j))] for j in range(n) ]
        print(" ".join(ss))

### 7.15 ナンバーリンク



格子グラフを返す関数 (gridGraph)



In [1]:
def gridGraph(m, n):
    vertices = [ (i,j) for i in range(m) for j in range(n) ]
    edges = []
    for (i,j) in vertices:
        for (k,l) in [(i,j+1), (i+1,j)]:
            if k in range(m) and l in range(n):
                edges.append(((i,j), (k,l)))
    return (vertices, edges)

ナンバーリンクのCSP (numberlink)



In [1]:
from cspsat import *
def numberlink(m, n, links, a=Bool("a"), d=Bool("d"), x=Var("x")):
    (vertices, edges) = gridGraph(m, n)
    def adj(v):
        return [ e[1-i] for e in edges for i in [0,1] if e[i] == v ]
    num = { v: h for h in links for v in links[h] }
    for v in vertices:
        yield ["int", x(v), 1, max(links.keys())]
        h = num.get(v, 0)
        if h:
            yield ["==", x(v), h]
            (inDeg, outDeg) = (0, 1) if v == links[h][0] else (1, 0)
        else:
            (inDeg, outDeg) = (d(v), d(v))
        yield ["==", ["+", *[ a(u,v) for u in adj(v) ]], inDeg]
        yield ["==", ["+", *[ a(v,u) for u in adj(v) ]], outDeg]
    for (u,v) in edges:
        yield ["imp", ["or", a(u,v), a(v,u)], ["==", x(u), x(v)]]

In [1]:
(m, n) = (5, 5)
links = { 1: [(0,4),(4,0)], 2: [(0,0),(3,2)], 3: [(1,2),(4,4)] }
x = Var("x")
for sol in solutionsCSP(numberlink(m, n, links)):
    for i in range(m):
        print([ sol[x((i,j))] for j in range(n) ])

### 7.16 スリザーリンク



単一閉路のCSP



In [1]:
from cspsat import *
def singleCycle(vertices, edges, minLen=None, maxLen=None, e=Bool("e"), a=Bool("a"), d=Bool("d"), r=Bool("r"), x=Var("x")):
    def adj(v):
        return [ edge[1-i] for edge in edges for i in [0,1] if edge[i] == v ]
    for (u,v) in edges:
        yield ["==", e(u,v), ["+", a(u,v), a(v,u)]]
    for v in vertices:
        yield ["==", ["+", *[ a(v,u) for u in adj(v) ]], d(v)]
        yield ["==", ["+", *[ a(u,v) for u in adj(v) ]], d(v)]
    yield ["eqK", [ r(v) for v in vertices ], 1]
    for v in vertices:
        yield ["int", x(v), 0, len(vertices)]
        yield ["equ", d(v), [">", x(v), 0]]
    for v in vertices:
        for u in adj(v):
            yield ["imp", ["and", r(v), a(v,u)], ["==", x(u), 1]]
            yield ["imp", ["and", ~r(v), a(v,u)], ["==", ["+", x(v), 1], x(u)]]
    for v in vertices:
        if minLen:
            yield ["imp", r(v), [">=", x(v), minLen]]
        if maxLen:
            yield ["imp", r(v), ["<=", x(v), maxLen]]
    if minLen:
        yield ["geK", [ d(v) for v in vertices ], minLen]
    if maxLen:
        yield ["leK", [ d(v) for v in vertices ], maxLen]

スリザーリンクのCSP



In [1]:
from cspsat.examples.graph import gridGraph
from itertools import product
def slitherlink(clues, e=Bool("e")):
    def surrounds(i, j):
        es = [ ((i,j),(i,j+1)), ((i,j),(i+1,j)), ((i,j+1),(i+1,j+1)), ((i+1,j),(i+1,j+1)) ]
        return [ e(u,v) for (u,v) in es ]
    (m, n) = (len(clues), len(clues[0]))
    (vertices, edges) = gridGraph(m+1, n+1)
    yield from singleCycle(vertices, edges, e=e)
    for (i,j) in product(range(m), range(n)):
        h = clues[i][j]
        if h <= 4:
            yield ["eqK", surrounds(i, j), h]

In [1]:
def showSlitherlink(clues, sol, e=Bool("e")):
    (m, n) = (len(clues), len(clues[0]))
    for i in range(m+1):
        line = "+"
        for j in range(n):
            line += "---+" if sol[e((i,j),(i,j+1))] else "   +"
        print(line)
        if i == m:
            continue
        line = ""
        for j in range(n+1):
            line += "|" if sol[e((i,j),(i+1,j))] else "."
            if j == n:
                continue
            h = clues[i][j]
            line += f" {h} " if h <= 4 else "   "
        print(line)

In [1]:
sl1 = [
    [0, 1, 9, 3],
    [9, 9, 2, 9]
]
for sol in solutionsCSP(slitherlink(sl1)):
    showSlitherlink(sl1, sol)

#### 練習問題 7.16.1: スリザーリンクの問題生成



In [1]:
import random
def uniqSol(csp):
    sols = list(solutionsCSP(csp, num=2))
    return len(sols) == 1
def generateSlitherlink(clues):
    (m, n) = (len(clues), len(clues[0]))
    if not uniqSol(slitherlink(clues)):
        print("Bad clues")
    positions = [ (i,j) for i in range((m+1)//2) for j in range(n) ]
    while positions:
        k = random.randrange(0, len(positions))
        (i,j) = positions.pop(k)
        (old1, old2) = (clues[i][j], clues[m-i-1][n-j-1])
        (clues[i][j], clues[m-i-1][n-j-1]) = (9, 9)
        if not uniqSol(slitherlink(clues)):
            (clues[i][j], clues[m-i-1][n-j-1]) = (old1, old2)
    return clues

In [1]:
clues = [ [2,2,3], [2,3,2], [2,2,2], [2,3,2], [3,2,2] ]
print(generateSlitherlink(clues))

### 7.17 ライフゲーム



ライフゲームのCSP



In [1]:
from cspsat import *
from itertools import product
def life(steps, m, n, x=Bool("x")):
    def surrounds(i, j):
        kls = [ kl for kl in product([i-1,i,i+1], [j-1,j,j+1]) if kl != (i,j) ]
        return [ (k,l) for (k,l) in kls if 0 <= k < m and 0 <= l < n ]
    def state(t):
        yield from [ ~x(t,i,j) for i in [0,m-1] for j in range(n) ]
        yield from [ ~x(t,i,j) for i in range(m) for j in [0,n-1] ]
    def transition(t):
        for (i,j) in product(range(m), range(n)):
            s = Var()
            yield ["int", s, 0, 8]
            yield ["==", s, ["+", *[ x(t,k,l) for (k,l) in surrounds(i, j) ]]]
            yield ["equ", x(t+1,i,j), ["or", ["==", s, 3], ["and", x(t,i,j), ["==", s, 2]]]]

    yield from state(0)
    for t in range(steps):
        yield from state(t+1)
        yield from transition(t)

tステップ目が与えられたパターンになる条件を生成する関数



In [1]:
def lifePattern(t, pattern, x=Bool("x")):
    for (i,row) in enumerate(pattern):
        for (j,p) in enumerate(row):
            yield ~x(t,i,j) if p == "." else x(t,i,j)

ライフゲームの解を表示する関数



In [1]:
def showLife(sol, steps, m, n, x=Bool("x")):
    for t in range(steps+1):
        c = 0
        for i in range(m):
            ss = [ sol[x(t,i,j)] for j in range(n) ]
            c += sum(ss)
            print("".join([ "#" if s else "." for s in ss ]))
        print(f"Step {t}: {c} cells")
        print()

例題「ライフゲーム」の解を探索



In [1]:
goal = [
    ".................",
    ".................",
    ".................",
    "...###.###.###...",
    "...#...#.#..#....",
    "...###.###..#....",
    ".....#.#.#..#....",
    "...###.#.#..#....",
    ".................",
    ".................",
    ".................",
]
(m, n) = (len(goal), len(goal[0]))
steps = 2
csp = [*life(steps, m, n), *lifePattern(steps, goal)]
for sol in solutionsCSP(csp):
    showLife(sol, steps, m, n)

In [1]:
def lifeLim(m, n, limits=[], x=Bool("x")):
    for (t,(lb,ub)) in enumerate(limits):
        xx = [ x(t,i,j) for i in range(m) for j in range(n) ]
        if isinstance(lb, int):
            yield ["geK", xx, lb]
        if isinstance(ub, int):
            yield ["leK", xx, ub]

In [1]:
(m, n) = (len(goal), len(goal[0]))
steps = 2
limits = [(None,30)]
csp = [*life(steps, m, n), *lifePattern(steps, goal), *lifeLim(m, n, limits)]
for sol in solutionsCSP(csp, command="bin/kissat"):
    showLife(sol, steps, m, n)

#### 練習問題 7.17.1: 増大するパターンを探す



In [1]:
(m, n) = (9, 9)
steps = 5
limits = [ (None, None) for t in range(steps+1) ]
limits[0] = (None, 5)
limits[steps] = (20, None)
csp = [*life(steps, m, n), *lifeLim(m, n, limits)]
for sol in solutionsCSP(csp):
    showLife(sol, steps, m, n)

#### 練習問題 7.17.2: 宇宙船



In [1]:
from itertools import product
def lifeMove(m, n, t1, t2, di, dj, x=Bool("x")):
    for (i,j) in product(range(m), range(n)):
        if i+di in range(m) and j+dj in range(n):
            yield ["equ", x(t2,i+di,j+dj), x(t1,i,j)]
        else:
            yield ~x(t1,i,j)

In [1]:
(m, n) = (7, 9)
steps = 4
limits = [ (1,None) for t in range(steps+1) ]
csp = [*life(steps, m, n), *lifeLim(m, n, limits), *lifeMove(m, n, 0, steps, 0, 2)]
for sol in solutionsCSP(csp):
    showLife(sol, steps, m, n)

In [1]:
(m, n) = (8, 18)
steps = 3
limits = [ (1,None) for t in range(steps+1) ]
csp = [*life(steps, m, n), *lifeLim(m, n, limits), *lifeMove(m, n, 0, steps, 1, 0)]
for sol in solutionsCSP(csp):
    showLife(sol, steps, m, n)

### 7.18 倉庫番



In [1]:
board = [
    "######",
    "# .###",
    "#  ###",
    "#*@  #",
    "#  $ #",
    "#  ###",
    "######"
]

In [1]:
def isWall(i, j, board): return board[i][j] == "#"
def isPlayer(i, j, board): return board[i][j] in "@+"
def isBox(i, j, board): return board[i][j] in "$*"
def isGoal(i, j, board): return board[i][j] in ".+*"

倉庫番のCSP



In [1]:
from cspsat import *
def sokoban(board, steps, p=Bool("p"), b=Bool("b"), g=Bool("g"), di=Var("di"), dj=Var("dj")):
    (m, n) = (len(board), len(board[0]))
    floor = { (i,j) for i in range(m) for j in range(n) if not isWall(i, j, board) }
    boxes = [ (i,j) for (i,j) in floor if isBox(i, j, board) ]
    goals = [ (i,j) for (i,j) in floor if isGoal(i, j, board) ]
    def state(t):
        yield ["eqK", [ p(t,i,j) for (i,j) in floor ], 1]
        yield ["eqK", [ b(t,i,j) for (i,j) in floor ], len(boxes)]
        yield ["equ", g(t), ["and", *[ b(t,i,j) for (i,j) in goals ]]]
    def initial(t):
        for (i,j) in floor:
            if isPlayer(i, j, board):
                yield p(t,i,j)
            elif isBox(i, j, board):
                yield b(t,i,j)
    def d0(i, j):
        return [ (k,l) for (k,l) in [(0,0),(-1,0),(1,0),(0,-1),(0,1)] if (i-k,j-l) in floor ]
    def d1(i, j):
        return [ (k,l) for (k,l) in [(-1,0),(1,0),(0,-1),(0,1)] if (i-k,j-l) in floor and (i-2*k,j-2*l) in floor ]
    def transition(t):
        yield ["int", di(t), -1, 1]
        yield ["int", dj(t), -1, 1]
        yield ["or", ["==", di(t), 0], ["==", dj(t), 0]]
        yield ["imp", g(t), ["and", ["==", di(t), 0], ["==", dj(t), 0]]]
        for (i,j) in floor:
            c = ["or"]
            for (k,l) in d0(i, j):
                c.append(["and", ["==", di(t), k], ["==", dj(t), l], p(t,i-k,j-l)])
            yield ["equ", p(t+1,i,j), c]
        for (i,j) in floor:
            c = ["or"]
            for (k,l) in d1(i, j):
                c.append(["and", b(t,i-k,j-l), p(t,i-2*k,j-2*l), p(t+1,i-k,j-l)])
            yield ["equ", b(t+1,i,j), ["or", ["and", ~b(t,i,j), c], ["and", b(t,i,j), ~p(t+1,i,j)]]]

    yield from state(0)
    yield from initial(0)
    for t in range(steps):
        yield from state(t+1)
        yield from transition(t)
    yield ["or", *[ g(t) for t in range(steps+1) ]]

In [1]:
def showSolution(sol, t, board, p=Bool("p"), b=Bool("b"), g=Bool("g")):
    (m, n) = (len(board), len(board[0]))
    floor = { (i,j) for i in range(m) for j in range(n) if not isWall(i, j, board) }
    player = [ (i,j) for (i,j) in floor if sol[p(t,i,j)] ][0]
    boxes = [ (i,j) for (i,j) in floor if sol[b(t,i,j)] ]
    goal = sol[g(t)]
    print(f"Step = {t}")
    print(f"goal = {goal}, player = {player}, boxes = {boxes}")
    for i in range(len(board)):
        s = []
        for j in range(len(board[i])):
            if isWall(i, j, board):
                c = "#"
            elif sol[p(t,i,j)]:
                c = "+" if isGoal(i, j, board) else "@"
            elif sol[b(t,i,j)]:
                c = "*" if isGoal(i, j, board) else "$"
            else:
                c = "." if isGoal(i, j, board) else " "
            s.append(c)
        print("".join(s))
    print()

In [1]:
steps = 33
for sol in solutionsCSP(sokoban(board, steps)):
    for t in range(0, steps+1):
        showSolution(sol, t, board)

#### 練習問題 7.18.1: 1プッシュを1ステップとする



倉庫番のCSP (箱を1回押す動作を1ステップとする)



In [1]:
from cspsat import *
def sokobanP(board, steps):
    (m, n) = (len(board), len(board[0]))
    floor = { (i,j) for i in range(m) for j in range(n) if not isWall(i, j, board) }
    edges = [ ((i,j),(i+di,j+dj)) for (i,j) in floor for (di,dj) in [(1,0),(0,1)] if (i+di,j+dj) in floor ]
    boxes = [ (i,j) for (i,j) in floor if isBox(i, j, board) ]
    goals = [ (i,j) for (i,j) in floor if isGoal(i, j, board) ]
    (p, r, b, g) = (Bool("p"), Bool("r"), Bool("b"), Bool("g"))
    (di, dj) = (Var("di"), Var("dj"))
    def state(t):
        yield ["eqK", [ p(t,i,j) for (i,j) in floor ], 1]
        yield ["eqK", [ b(t,i,j) for (i,j) in floor ], len(boxes)]
        yield ["equ", g(t), ["and", *[ b(t,i,j) for (i,j) in goals ]]]
    def initial(t):
        for (i,j) in floor:
            if isPlayer(i, j, board):
                yield p(t,i,j)
            elif isBox(i, j, board):
                yield b(t,i,j)
    def d0(i, j):
        return [ (k,l) for (k,l) in [(0,0),(-1,0),(1,0),(0,-1),(0,1)] if (i-k,j-l) in floor ]
    def d1(i, j):
        return [ (k,l) for (k,l) in [(-1,0),(1,0),(0,-1),(0,1)] if (i-k,j-l) in floor and (i-2*k,j-2*l) in floor ]
    def reachability(t):
        (a, inD, outD) = (Bool(), Bool(), Bool())
        for (u,v) in edges:
            yield ["leK", [a(t,u,v), a(t,v,u)], 1]
        for v in floor:
            (i,j) = v
            us = [ e[1-i] for e in edges for i in [0,1] if e[i] == v ]
            yield ["==", ["+", *[ a(t,u,v) for u in us ]], inD(t,i,j)]
            yield ["==", ["+", *[ a(t,v,u) for u in us ]], outD(t,i,j)]
            yield ["imp", p(t,i,j), ["or", r(t,i,j), ["and", ~inD(t,i,j), outD(t,i,j)]]]
            yield ["imp", r(t,i,j), ["or", p(t,i,j), ["and", inD(t,i,j), ~outD(t,i,j)]]]
            yield ["imp", ["and", ~p(t,i,j), ~r(t,i,j)], ["==", inD(t,i,j), outD(t,i,j)]]
            yield ["imp", b(t,i,j), ["and", ~inD(t,i,j), ~outD(t,i,j)]]
    def transition(t):
        yield ["int", di(t), -1, 1]
        yield ["int", dj(t), -1, 1]
        yield ["or", ["==", di(t), 0], ["==", dj(t), 0]]
        yield ["imp", g(t), ["and", ["==", di(t), 0], ["==", dj(t), 0]]]
        yield from reachability(t)
        for (i,j) in floor:
            c = ["or"]
            for (k,l) in d0(i, j):
                c.append(["and", ["==", di(t), k], ["==", dj(t), l], r(t,i-k,j-l)])
            yield ["equ", p(t+1,i,j), c]
        for (i,j) in floor:
            c = ["or"]
            for (k,l) in d1(i, j):
                c.append(["and", b(t,i-k,j-l), r(t,i-2*k,j-2*l), p(t+1,i-k,j-l)])
            yield ["equ", b(t+1,i,j), ["or", ["and", ~b(t,i,j), c], ["and", b(t,i,j), ~p(t+1,i,j)]]]

    yield from state(0)
    yield from initial(0)
    for t in range(steps):
        yield from state(t+1)
        yield from transition(t)
    yield ["or", *[ g(t) for t in range(steps+1) ]]

In [1]:
steps = 8
for sol in solutionsCSP(sokobanP(board, steps)):
    for t in range(0, steps+1):
        showSolution(sol, t, board)

#### 練習問題 7.18.2: ペグソリテア



In [1]:
from cspsat import *
def pegSolitaire(board0, board1, steps, x=Bool("x"), p=Bool("p")):
    (m, n) = (len(board0), len(board0[0]))
    vertices = [ (i,j) for i in range(m) for j in range(n) if board0[i][j] != " " ]
    dirs = [(-1,0), (0,-1), (0,1), (1,0)]
    def adj2(u):
        (i,j) = u
        return [ ((i+di,j+dj),(i+2*di,j+2*dj)) for (di,dj) in dirs if (i+di,j+dj) in vertices and (i+2*di,j+2*dj) in vertices ]
    def setBoard(t, board):
        for u in vertices:
            (i,j) = u
            if board[i][j] == ".":
                yield ~x(t,u)
            elif board[i][j] == "#":
                yield x(t,u)
    def transition(t):
        yield ["eqK", [ p(t,u,w) for u in vertices for (_,w) in adj2(u) ], 1]
        for u in vertices:
            for (v,w) in adj2(u):
                yield ["imp", p(t,u,w), ["and", x(t,u), x(t,v), ~x(t,w)]]
        for u in vertices:
            f = [ p(t,v1,v3) for v1 in vertices for (v2,v3) in adj2(v1) if u in [v1,v2,v3] ]
            yield ["equ", x(t+1,u), ["xor", x(t,u), ["or", *f]]]

    yield from setBoard(0, board0)
    yield from setBoard(steps, board1)
    for t in range(steps):
        yield from transition(t)

In [1]:
board0 = [
    "  ###  ",
    "  ###  ",
    "#######",
    "###.###",
    "#######",
    "  ###  ",
    "  ###  ",
]
board1 = [
    "  ...  ",
    "  ...  ",
    ".......",
    "...#...",
    ".......",
    "  ...  ",
    "  ...  ",
]
(m, n) = (len(board0), len(board0[0]))
steps = "".join(board0).count("#") - "".join(board1).count("#")
x = Bool("x")
for sol in solutionsCSP(pegSolitaire(board0, board1, steps)):
    for t in range(steps+1):
        for i in range(m):
            ss = [ " " for j in range(n) ]
            for j in range(n):
                if board0[i][j] != " ":
                    ss[j] = "#" if sol[x(t,(i,j))] else "."
            print("".join(ss))
        print(f"Step {t}")
        print()
    print(status())

### 7.19 エニグマ暗号



#### 7.19.7 プログラム



定数



In [1]:
from cspsat import *
def str2ints(s): return [ ord(c) - ord("A") for c in s ]
n = 26
k = 3
rotorData = {
    1: str2ints("EKMFLGDQVZNTOWYHXUSPAIBRCJ"),
    2: str2ints("AJDKSIRUXBLHWTMCQGZNPYFVOE"),
    3: str2ints("BDFHJLCPRTXVZNYEIWGAKMUSQO")
}
reflectorData = str2ints("YRUHQSLDPXNGOKMIEBFZCWVJAT")
(r, p, s, w) = (Var("r"), Var("p"), Var("s"), Var("w"))

ローターとプラグボード設定



In [1]:
from itertools import combinations, permutations
def enigmaDef(l=3, useW=True):
    for j in range(3):
        yield ["int", r(j), 1, k]
    yield ["alldifferent", *[ r(j) for j in range(3) ]]
    for j in range(3):
        yield ["int", p(j), 0, n-1]
    for a in range(n):
        yield ["int", s(a), 0, n-1]
    for (a,b) in combinations(range(n), 2):
        yield ["equ", ["==", s(a), b], ["==", s(b), a]]
    q = Bool()
    for a in range(n):
        yield ["equ", q(a), ["!=", s(a), a]]
    yield ["leK", [ q(a) for a in range(n) ], 2*l]
    if useW:
        yield from enigmaDefW()

In [1]:
def enigmaDefW():
    for a in range(n):
        yield ["int", w(a), 0, n-1]
    ww = {}
    for (j0,j1) in permutations(range(1,k+1), 2):
        r0 = rotorData[j0]
        r0x = dict((b,a) for (a,b) in enumerate(r0))
        r1 = rotorData[j1]
        r1x = dict((b,a) for (a,b) in enumerate(r1))
        for p0 in range(n):
            for p1 in range(n):
                for a in range(n):
                    b = (r1[(a+p1)%n]-p1)%n
                    b = (r0[(b+p0)%n]-p0)%n
                    b = reflectorData[b]
                    b = (r0x[(b+p0)%n]-p0)%n
                    b = (r1x[(b+p1)%n]-p1)%n
                    if (a,b) not in ww:
                        ww[(a,b)] = []
                    ww[(a,b)].append(["and", ["==", r(0), j0], ["==", r(1), j1], ["==", p(0), p0], ["==", p(1), p1]])
    for (a,b) in ww.keys():
        yield ["equ", ["==", w(a), b], ["or", *ww[(a,b)]]]

プラグボード，ローター，リフレクターの置換



In [1]:
def plugboard(x, z):
    for a in range(n):
        yield ["equ", ["==", x, a], ["==", z, s(a)]]
def reflector(x, z):
    for a in range(n):
        yield ["equ", ["==", x, a], ["==", z, reflectorData[a]]]
def rotor(i, j, x, z):
    dp = i if j == 2 else 0
    y = Var()
    yield ["int", y(0), 0, n-1]
    yield ["int", y(1), 0, n-1]
    yield ["==", y(0), ["mod", ["+", x, p(j), dp], n]]
    yield ["==", y(1), ["mod", ["+", z, p(j), dp], n]]
    for d in rotorData.keys():
        for a in range(n):
            yield ["imp", ["==", r(j), d], ["equ", ["==", y(0), a], ["==", y(1), rotorData[d][a]]]]

i文字目の置換



In [1]:
def encipher(i, x, z, useW=True):
    y = Var()
    if useW:
        for t in range(4):
            yield ["int", y(t), 0, n-1]
        yield from plugboard(x, y(0))
        yield from rotor(i, 2, y(0), y(1))
        for a in range(n):
            yield ["equ", ["==", y(1), a], ["==", y(2), w(a)]]
            yield ["equ", ["==", y(1), w(a)], ["==", y(2), a]]
        yield from rotor(i, 2, y(3), y(2))
        yield from plugboard(y(3), z)
    else:
        for t in range(8):
            yield ["int", y(t), 0, n-1]
        yield from plugboard(x, y(0))
        yield from rotor(i, 2, y(0), y(1))
        yield from rotor(i, 1, y(1), y(2))
        yield from rotor(i, 0, y(2), y(3))
        yield from reflector(y(3), y(4))
        yield from rotor(i, 0, y(5), y(4))
        yield from rotor(i, 1, y(6), y(5))
        yield from rotor(i, 2, y(7), y(6))
        yield from plugboard(y(7), z)

In [1]:
def chr2int(c): return ord(c) - ord("A")
def ints2str(ii): return "".join([ chr(ord("A")+i) for i in ii ])
def enigmaSim(rotors, rings, positions, plugs, xx, z=Var("z"), useW=True):
    yield from enigmaDef(useW=useW)
    for j in range(3):
        yield ["==", r(j), rotors[j]]
    for j in range(3):
        yield ["==", p(j), (chr2int(positions[j])-chr2int(rings[j]))%n]
    pp = dict((chr2int(cs[k]),chr2int(cs[1-k])) for cs in plugs for k in [0,1])
    for a in range(n):
        yield ["==", s(a), pp[a] if a in pp else a]
    for (i,x) in enumerate(xx):
        yield ["int", z(i), 0, n-1]
        yield from encipher(i+1, chr2int(x), z(i), useW=useW)

In [1]:
z = Var("z")
xx = "XYZXYZ"
for sol in solutionsCSP(enigmaSim([2,1,3], "SAT", "ABC", ["KO","BE","JP"], xx, useW=False)):
    print(ints2str([ sol[z(i)] for i in range(len(xx)) ]))

#### 7.19.8 エニグマ暗号の解読



In [1]:
def enigmaBreak(plain, cipher):
    xx = str2ints(plain)
    zz = str2ints(cipher)
    yield from enigmaDef()
    for i in range(len(plain)):
        yield from encipher(i+1, xx[i], zz[i])
def enigmaSol(sol):
    rs = [ sol[r(j)] for j in range(3) ]
    ps = ints2str([ sol[p(j)] for j in range(3) ])
    ss = []
    for a in range(n):
        if sol[s(a)] > a:
            ss.append(ints2str([a, sol[s(a)]]))
    print(f"Rotors: {rs}")
    print(f"Positions: {ps}")
    print(f"Plugboard: {ss}")

In [1]:
for sol in solutionsCSP(enigmaBreak("WEATHERXREPORT", "JZGUAOVYSHBFKH")):
    enigmaSol(sol)

In [1]:
def enigmaKey(rotors, plugs, zz, x=Var("x"), useW=True):
    yield from enigmaDef(useW=useW)
    for j in range(3):
        yield ["==", r(j), rotors[j]]
    pp = dict((chr2int(cs[k]),chr2int(cs[1-k])) for cs in plugs for k in [0,1])
    for a in range(n):
        yield ["==", s(a), pp[a] if a in pp else a]
    for (i,z) in enumerate(zz):
        yield ["int", x(i), 0, n-1]
        yield from encipher(i+1, x(i), chr2int(z), useW=useW)
    yield ["==", x(0), x(3)]
    yield ["==", x(1), x(4)]
    yield ["==", x(2), x(5)]

In [1]:
x = Var("x")
zz = "ZQWMSH"
for sol in solutionsCSP(enigmaKey([2,1,3], ["BE","JP","KO"], zz, useW=False), num=0):
    enigmaSol(sol)
    print(ints2str([ sol[x(i)] for i in range(len(xx)) ]))

##### 練習問題 7.19.1: エニグマシミュレーター



In [1]:
from cspsat.examples.enigma_sim import *
plain = "WEATHERXREPORT"
enigma = EnigmaSim()
enigma.setRotors([2,1,3])
enigma.setRingPositions("SAT")
enigma.setStartPositions("XYZ")
enigma.setPlugboard(["KO","BE","JP"])
cipher = enigma.encipherStr(plain)
print(cipher)

##### 練習問題 7.19.2: 中央ローターの回転



In [1]:
from cspsat.examples.enigma import *
xx = "WEATHERXREPORTXZEROSIXZEROZERO"
zz = "JZGUAOVYSHBFKHLRVVKGMUMNSXDUJR"
enigma = Enigma()
for sol in solutionsCSP(enigma.enigmaBreak(xx, zz), num=0, command="bin/kissat"):
    enigma.showSol(sol)
    print()